In [1]:
#import libs
import matplotlib.pyplot as plt
import numpy as np
import pyaudio
import struct
import os
from scipy.fftpack import fft
from tkinter import TclError
import time


%matplotlib tk 
#opens up new window for live graphing

CHUNK = 4096 #size of each audio packet
FORMAT = pyaudio.paInt16 #bit size of audio packet input
CHANNELS = 1 #mono channel microphone
RATE = 44100 #44.1 kHz recording freq

In [ ]:
fig, az = plt.subplots(3, figsize=(12,5))
Barx = np.arange(15)
Barline, = az.plot(Barx, np.random.rand(16), '-')

In [2]:
#INITIALIZE PYAUDIO STREAM
PAud = pyaudio.PyAudio()
stream = PAud.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
    )

#SET UP PLOTS
fig, (ax, ay, az) = plt.subplots(3, figsize=(12,8))

#X-Axes
Audx = np.arange(0, 2*CHUNK, 2)
FFTx = np.linspace(0, RATE, CHUNK)
Barx = np.arange(16)

#Line objs
Audline, = ax.plot(Audx, np.random.rand(CHUNK), '-')
FFTline, = ay.semilogx(FFTx, np.random.rand(CHUNK), '-')
Barline, = az.plot(Barx, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], '-')

#make the plot look nice
ax.set_title('Recorded Audio')
ax.set_ylabel('Magnitude')
ax.set_ylim(0,255)
ax.set_xlim(0,2*CHUNK)
plt.setp(ax, xticks=[0, CHUNK, 2*CHUNK], yticks=[0,128,255])
ay.set_xlim(20, RATE/2)
az.set_ylim(0,1)


#LOOP FOR AUDIO
#now = time.time()
#THE 'DO STUFF' PART OF THE CODE
while True:
    #Gather data and unpack into integers
    data = stream.read(CHUNK)
    Idata=struct.unpack(str(2*CHUNK) + 'B', data)
        
    #place into array and offset accordingly for smooth sound wave
    Ndata = np.array(Idata, dtype='b')[::2] + 128
    Audline.set_ydata(Ndata)
        
    #do the FFT magic
    FFTy = fft(Ndata)
    savedata = np.abs(FFTy[0:CHUNK] * 4 / (256 * CHUNK))
    FFTline.set_ydata(savedata)
    
    #Bar graph
    newdata = FFTline.get_ydata()[1:]
    #create bins
    bins = [
        newdata[0:4], #0-44
        newdata[5:11], #55-121
        newdata[12:18], #121-198
        newdata[19:26], #198-286
        newdata[27:34], #286-374
        newdata[35:44], #374-484
        newdata[45:58], #484-638
        newdata[59:70], #638-770
        newdata[71:95], #770-1.05k
        newdata[96:115], #1.05k-1.27k
        newdata[116:180], #1.27k-2k
        newdata[181:240], #2k-2.6k
        newdata[241:340], #2.6k-3.8k
        newdata[341:454], #3.8k-5k
        newdata[455:910], #5k - 10k
        newdata[911:2000] #10k - 20k
    ]
    
    #get max 3 values and sum
    y_data = []

    #Get largest values from the bins
    for i in range(len(bins)):
        curBin = bins[i]
        binData = curBin[np.argsort(curBin)[-3:]] #Get highest 3 values
        binData = binData**3 * 54 #Log scaling 
        y_data.append(np.sum(binData))
        
    Barline.set_ydata(y_data)
        
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
            
    except TclError:
        print ('Goodbye!')
        break

Goodbye!


In [4]:
#ALL CELLS BELOW ARE PURELY FOR TESTING, DO NOT CONTRIBUTE TO PROGRAM AT ALL
############################################################################

#establishing bin frequencies, all comments are ranges
bins = [
    newdata[0:4], #0-44
    newdata[5:11], #55-121
    newdata[12:18], #121-198
    newdata[19:26], #198-286
    newdata[27:34], #286-374
    newdata[35:44], #374-484
    newdata[45:58], #484-638
    newdata[59:70], #638-770
    newdata[71:95], #770-1.05k
    newdata[96:115], #1.05k-1.27k
    newdata[116:180], #1.27k-2k
    newdata[181:240], #2k-2.6k
    newdata[241:340], #2.6k-3.8k
    newdata[341:454], #3.8k-5k
    newdata[455:910], #5k - 10k
    newdata[911:2000] #10k - 20k
]

y_pos = np.arange(16)
y_data = []

#Get largest values from the bins
for i in range(len(bins)):
    curBin = bins[i]
    binData = curBin[np.argsort(curBin)[-3:]]
    y_data.append(np.sum(binData))

print (y_data)

plt.bar(y_pos, y_data, align='center', alpha=0.5)
plt.xticks(y_pos)
plt.show()

[0.09303023401217993, 0.5223755982148592, 0.1943946362277516, 0.0667295841440441, 0.02745150454506514, 0.018372979652865405, 0.014841445942775046, 0.011813486242540822, 0.009568827415996245, 0.007172716095468625, 0.0063974475225821135, 0.00446618268119259, 0.004017223571767641, 0.003244871436783997, 0.00295910490895097, 0.002461979779029038]


In [80]:
#Splitting up into bins and seeing what looks good

#RUN BELOW COMMAND ONCE TO ENSURE DATA IS USEABLE
#savedata = np.append(savedata, 0)

print (len(savedata), savedata)
bins = np.split(savedata, 16)
y_pos = np.arange(len(bins))
#print (bins)
#binData = np.arange(len(bins))

for i in range(len(bins)):
    curBin = bins[i]
    binData = curBin[np.argsort(curBin)[-3:]]
    print (binData)
    binData = np.sum(binData)
    print (binData)

4096 [1.99248505 0.00855806 0.00800033 ... 0.00996239 0.00800033 0.00855806]
[0.29510539 0.57430302 1.99248505]
2.8618934535407505
[0.0010205  0.00107893 0.00111861]
0.0032180414863896517
[0.0006997  0.00070572 0.00090323]
0.0023086441231129394
[0.00069316 0.00071036 0.00074678]
0.002150310683737921
[0.00065526 0.00069035 0.00072371]
0.0020693105093441094
[0.00065285 0.00068795 0.00083162]
0.0021724222036467416
[0.00068222 0.00068306 0.00068993]
0.002055209375679957
[0.00051574 0.00051611 0.00053908]
0.001570931879299717
[0.00051574 0.00051611 0.00053908]
0.001570931879299717
[0.00068222 0.00068306 0.00068993]
0.002055209375679957
[0.00065285 0.00068795 0.00083162]
0.0021724222036467416
[0.00065526 0.00069035 0.00072371]
0.0020693105093441094
[0.00069316 0.00071036 0.00074678]
0.002150310683737921
[0.0006997  0.00070572 0.00090323]
0.0023086441231129394
[0.0010205  0.00107893 0.00111861]
0.0032180414863896517
[0.14673601 0.29510539 0.57430302]
1.0161444150655539
